In [ ]:
# TEXT AND IMAGE EXTRACTION



import os
import fitz

def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def extract_images_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    images_directory = "images"  
    ensure_directory_exists(images_directory) 
    for i, page in enumerate(doc):
        image_list = page.get_images(full=True)
        # get the XREF of the image
        for image_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            # extract the image bytes
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            # save it to local disk
            with open(f"images/page_{i}_img_{image_index}.png", "wb") as image_file:
                image_file.write(image_bytes)
        
        
pdf_path = r"C:\Users\shrav\OneDrive\Desktop\ML projects\AI data science teaching assistant\data_science_interview_questions.pdf"
extracted_text = extract_text_from_pdf(pdf_path)
extract_images_from_pdf(pdf_path)

text_file_path = "extracted_text.txt"
with open(text_file_path, "w", encoding="utf-8") as text_file:
    text_file.write(extracted_text)
print(f"Extracted text has been saved to {text_file_path}")




    

In [11]:
#STRUCTURING DATA in .csv FILE

import pandas as pd
import os
import re
import pathlib

qa_text_file_path = "C:/Users/shrav/OneDrive/Desktop/ML projects/AI data science teaching assistant/extracted_text.txt"
images_directory = "C:/Users/shrav/OneDrive/Desktop/ML projects/AI data science teaching assistant/images"


def extract_qa_pairs_from_text(file_path):
    qa_pairs = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines  = file.readlines()
    
    current_qa = {"Question": "", "Answer":""}
    
    for i, line in enumerate(lines):
        line = line.strip()
        if line.startswith('Q'):
            if current_qa["Question"]:
                qa_pairs.append(current_qa)
                current_qa={"Question":"", "Answer":""}
          
            current_qa["Question"]= line
            for j in range(i+1, len(lines)):
                next_line = lines[j].strip()
                if next_line.startswith("A"):
                    break
                current_qa["Question"]+= next_line+" "
            
                
        elif line.startswith('A'):
            if line.startswith("A"):
                current_qa["Answer"]+= line[0:].strip()+" "
            for j in range (i+1, len(lines)):
                next_line = lines[j].strip()
                if next_line.startswith("Q"):
                    break
                current_qa["Answer"]+= next_line+" "
            
    
    if current_qa["Question"]:
        qa_pairs.append(current_qa)
        
    return qa_pairs


"""



def associate_images_with_qa_pairs(qa_pairs, images_dir):
    for qa_pair in qa_pairs:
        q_number = re.search(r'Q(\d+)', qa_pair["Question"]).group(1)
        
        matched_images = []
        for image_file in os.listdir(images_directory):
            if re.match(f'Q{q_number}_img_\\d+.png', image_file):
                image_path = os.path.join(images_dir, image_file)
                image_url = pathlib.Path(image_path).as_uri()
                matched_images.append(image_url)
        
        qa_pair["Images"] = matched_images
"""

def remove_qa_identifiers(qa_pairs):
    for qa_pair in qa_pairs:
        qa_pair["Question"]= re.sub(r'Q\d+\.\s*','',qa_pair["Question"]).strip()
        qa_pair["Answer"] = re.sub(r'^A\d+\.\s*', '', qa_pair["Answer"]).strip()


# Extract Q&A pairs from the text file and associate images
qa_pairs = extract_qa_pairs_from_text(qa_text_file_path)
#associate_images_with_qa_pairs(qa_pairs, images_directory)
remove_qa_identifiers(qa_pairs)

# Convert to DataFrame
df = pd.DataFrame(qa_pairs)
#df['Images'] = df['Images'].apply(lambda x: '; '.join(x))  # Join multiple images into a single string

#Save the DataFrame to a CSV file
output_csv_path = "C:/Users/shrav/OneDrive/Desktop/ML projects/AI data science teaching assistant/Structured_Data/QA_Data.csv"
df.to_csv(output_csv_path, index=False, encoding='utf-8')
print(f"Data has been organized and saved to {output_csv_path}.")


Data has been organized and saved to C:/Users/shrav/OneDrive/Desktop/ML projects/AI data science teaching assistant/Structured_Data/QA_Data.csv.
